<a href="https://colab.research.google.com/github/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/Chapter13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 13. 듀얼리티의 활용
## 13.1 들어가며

* 듀얼리티(duality)
  * 머신러닝을 통해 특정 도메인의 데이터 X를 받아 다른 도메인의 데이터 Y로 맵핑하는 함수를 근사
  * 대부분의 머신러닝 데이터셋은 두 도메인 사이의 데이터로 구성된다.(오디오, 텍스트)

  <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/chap13/image1.png?raw=1" width="50%" height="50%">


* 듀얼리티 = 두 도메인 사이의 관계(음성 인식, 음성 합성)
  * 대부분 머신러닝 문제가 가지는 듀얼리티 속성
  * 기계번역 : 각 도메인 데이터 사이에 정보량 차이가 거의 없다(장점)
  * 듀얼리티 적극적으로 활용하면 기계번역 고도화 가능하다.

* 데이터에 따른 듀얼리티 사례

|$D_1$ 도메인 | $D_1 \rightarrow D_2$ | $D_1 \leftarrow D_2$ | $D_2$ 도메인 |
| :-------- | :---------------------| :-------------------- |:---------- |
| 문장 | 기계번역 | 기계번역 | 문장 |
| 음성 신호 | 음성 인식 | 음성 합성 | 문장 |
| 이미지 | 이미지 분류 | 이미지 합성 | 클래스|
| 문서 | 요약 | 본문 생성 | 문장 |
| 문장 | 텍스트 분류 | 문장 생성 | 클래스 |

### 13.1.2 CycleGAN

* 번역 이외 듀얼리티, 영상 처리에 관한 논문 CycleGAN
* cycleGAN
  * 짝지어지지 않은 두 도메인의 이미지가 있을 때, X 도메인의 이미지를 Y 도메인의 이미지로 변환 방법
  * 사진의 전체 구조 유지하되 유명한 화가 모네 풍으로 바꾸거나 말을 얼룩말 / 민무늬 변경
  * 겨울 풍경을 여름 풍경으로 변화 가능
  * 예제  
    <img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/chap13/image2.jpg?raw=1" width="100%" height="50%">

* $X\rightarrow{Y}$ 와 $Y\rightarrow{X}$ : 모두 각각 생성자 $G, F$ 와 판별자 $D_x, D_y$ 를 가지고 있어서 최소/최대화 게임 수행
* $G$ : $x$ 를 입력으로 받아 $\hat{y}$ 으로 변환
* $D_Y$ : $\hat{y}$ 또는 $y$ 를 입력으로 받아 합성한 이미지인지 원래 $Y$ 도메인의 이미지인지 예측
* $F$ : $y$ 를 입력으로 받아 $\hat{x}$ 으로 변환
* $D_x$ : $\hat{x}$ 또는 $x$ 를 입력으로 받아 합성 유부를 판단

<img src="https://github.com/CUAI-Deeplearning/Pytorch-NLP-Study/blob/stats/%EA%B0%95%ED%98%B8%ED%98%84/asset/chap13/image3.png?raw=1" width="100%" height="50%">

* $\hat{X}, \hat{Y}$를 합성할 때 기존의 도메인 X, Y에 실제로 속하는 이미지처럼 만들어야 한다.
* $\hat{X}, \hat{Y}$에서 원래 데이터로 돌아올 수 있어야 한다.
$\rightarrow$ 기계번역에 적용

## 13.2 듀얼리티를 활용한 지도학습
* 듀얼 지도학습(DSL, Dual Supervised Learning)
  * 기존의 teacher-forcing 문제로 생기는 어려움을 해결할 때 강화학습 대신 듀얼리티에서 regularization term 이끌어내 해결한다.
  * 베이즈 정리에 따른 수식(항상 성립)
  $$P(y|x) = {P(x|y)P(y)} \over {P(x)}$$
  $$P(y|x)P(x) = P(x|y)P(y)$$

* 이 수식대로 훈련한 모델은 다음 수식을 만족한다.
$$P(x)P(y|x;\theta_{y\rightarrow x}) = P(y)P(x|y;\theta_{y\rightarrow x})
$$
$$ \text{where } (x,y) \sim \mathcal{B} = \{x_i,y_i\}_{i=1}^N$$

* 이 전제를 번역 훈련을 위한 목표에 적용한다.

$$objective1: \text{min}_{\theta_{x\rightarrow y}} {1 \over n} \sum_{i=1}^n \ell_1(f(x_i;\theta{x\rightarrow y}),y_i), $$

$$objective2: \text{min}_{\theta_{y\rightarrow x}} {1 \over n} \sum_{i=1}^n \ell_1(g(x_i;\theta{y\rightarrow x}),x_i), $$

$$s.t. P(x) P(y|x;\theta_{x\rightarrow y}) = P(y) P(x|y; \theta_{y\rightarrow x}), \forall x,y $$

* 해석
  * 목표 1 : 베이즈 정리에 따른 제약 조건 만족함과 동시에 $\ell_1$을 최소화
    * $\ell_1$ : 번역함수 f에 입력 $x_i$를 넣어 나온 반환값과 $y_i$ 사이 손실
  * 목표 2 : $\ell_2$를 번역함수 $g$에 대해 같은 작업 수행하고 최소화하여 만족시킨다.
    * $\ell_2$ : 번역함수 g에 입력 $x_i$를 넣어 나온 반환값과 $y_i$ 사이 손실


$\mathcal{L}_{duality} = |\left(\log \hat{P}(x) + \log P(y|x;\theta_{x\rightarrow y})\right) - \left(\log \hat{P}(y) + \log P(x|y;\theta_{y\rightarrow x})\right) |^2$

* 베이즈 정리에 따른 제약 조건의 양 변의 값의 차이를 최소화하는 MSE 손실 함수
* 동시에 훈련시키는 신경망 가중치 파라미터 $\rightarrow$ 각각 $\log P(y|x;\theta_{x\rightarrow y}),\log P(x|y;\theta_{y\rightarrow x}) $ 구한다.
* 단일 언어 코퍼스로 미리 훈련시킨 언어 모델 $\rightarrow \log \hat{P}(x), \log \hat{P}(y)$ 근사 가능하다.

* 기존의 손실 함수에 추가하여 동시에 최소화

$$ \theta_{x \rightarrow y} \leftarrow \theta_{x \rightarrow y} - \gamma \triangledown_{\theta_{x \rightarrow y}} {1 \over n} \sum_{i=1}^n [\ell_1(f(x_i;\theta_{x\rightarrow y}),y_i) + \lambda_{x \rightarrow y} \mathcal{L}_{duality}]
$$


$$ \theta_{y \rightarrow x} \leftarrow \theta_{y \rightarrow x} - \gamma \triangledown_{\theta_{y \rightarrow x}} {1 \over n} \sum_{i=1}^n [\ell_2(g(y_i;\theta_{y\rightarrow x}),x_i) + \lambda_{y \rightarrow x} \mathcal{L}_{duality}]$$


* $\lambda$를 통해 손실함수 내 비율 조절 가능
  * $\lambda$ 는 하이퍼 파라미터, 0.01일 때 가장 좋은 성능을 보인다.
  * DSL 방식이 기존의 teacher-forcing, MLE / MRT 보다 더 높은 성능을 보인다.
  * Regularization term을 추가하여 훈련 까다롭고 비효율적인 강화학습보다 나은 성능을 보인다.

### 13.2.1 파이토치 예제 코드

In [0]:
# simple_nmt와 dual_trainer.py
X2Y, Y2X = 0, 1

In [0]:
def _reordering(self, x, y, l):
  indice = l.topk(l.size(0))[1]